**Problem Statement :**  
Using the previous tables, Write a dask cudf accelerated code to get the `top 5 most sold items` on given day along with their units sold. And then get the transactions for those items(i.e. find the store number and all other information) 

---

In [2]:
from dask_cudf import read_csv, read_parquet, read_orc, DataFrame, concat
## Q1. Right Import
from dask_cudf import read_csv, read_parquet, read_orc, DataFrame, concat


In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

## Q2. Create a Cluster
cluster = LocalCUDACluster(dashboard_address=':8001')

## Q3. Create a Client
client = Client(cluster)
client

/conda/envs/rapids/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8001 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41251 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:46220 Dashboard: http://127.0.0.1:41251/status,Cluster Workers: 2 Cores: 2 Memory: 810.04 GB


---
#### Reading the cal_dim table from `/data/calendar.csv` stored in csv format

In [4]:
%%time
cal = read_csv('/data/calendar.csv')
cal['cal_dt'] = cal['cal_dt'].astype('datetime64[ms]')
cal['ly_date'] = cal['ly_date'].astype('datetime64[ms]')
cal['ly_comp_visit_dt'] = cal['ly_comp_visit_dt'].astype('datetime64[ms]')

CPU times: user 2.14 s, sys: 741 ms, total: 2.88 s
Wall time: 2.75 s


In [5]:
## Q4 Compute cal to cudf dataframe
cal = cal.compute()


In [6]:
cal

,cal_dt,ly_date,ly_comp_visit_dt,wm_yr_wk_nbr
0,2017-04-11,2016-04-11,2016-04-12,201711


---
#### Reading the store_visit_scan table from `/data/store_visit_scan_mask.parquet` stored in parquet format

In [7]:
%%time
svs = read_parquet('/data/store_visit_scan_mask.parquet')
svs['load_ts'] = svs['load_ts'].astype('datetime64[ms]')
svs['visit_dt'] = svs['visit_dt'].astype('datetime64[ms]')


CPU times: user 45.6 ms, sys: 12.5 ms, total: 58.1 ms
Wall time: 48.2 ms


In [8]:
svs

,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd
npartitions=7,,,,,,,,
,int64,int64,int64,int64,int64,datetime64[ms],datetime64[ms],object
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [ ]:
## Q5. How many partitions does the dask_cudf data frame has?
## A. 7

In [10]:
## Q6. Change the data farme from previous number of partitons to to 4 partitions

svs = svs.repartition(npartitions=4)

In [11]:
## Q7. Print the head to understand the DataFrame 
svs.head()

,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd
0,710103784,1099,8,877534402,0,2017-06-23 12:46:35,2017-04-11,WMT-US
1,710104362,1999,7,867174169,0,2017-06-23 12:46:35,2017-04-11,WMT-US
2,710101771,17377,6,738523462,0,2017-06-23 12:46:35,2017-04-11,WMT-US
3,710103327,9700,10,892230403,0,2017-06-23 12:46:35,2017-04-11,WMT-US
4,710101361,8737,4,851258830,0,2017-06-23 12:46:35,2017-04-11,WMT-US


In [12]:
## Q8. Print the dask DataFrame to understand the partitions
svs

,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd
npartitions=4,,,,,,,,
,int64,int64,int64,int64,int64,datetime64[ms],datetime64[ms],object
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


---
#### Reading the store_visit table from `/data/store_visit_mask.parquet` stored in parquet format

In [13]:
%%time

vst = read_parquet('/data/store_visit_mask.parquet')

## Q9. read parquet files with dask cudf
vst = read_parquet('/data/store_visit_mask.parquet')
vst['visit_dt'] = vst['visit_dt'].astype('datetime64[ms]')
vst['visit_hr'] = (vst.visit_tm / 100).astype('i2')
vst['visit_min'] = (vst.visit_tm%100).astype('i2')
vst = vst.drop(['visit_tm'], axis=1)

CPU times: user 1.03 s, sys: 47.2 ms, total: 1.08 s
Wall time: 1.03 s


In [14]:
vst

,store_nbr,visit_nbr,visit_dt,op_cmpny_cd,visit_hr,visit_min
npartitions=8,,,,,,
,int64,int64,datetime64[ms],object,int16,int16
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [16]:
## Q10. Change the data farme from previous number of partitons to to 4 partitions
vst = vst.repartition(npartitions=4)


In [17]:
vst.head()

,store_nbr,visit_nbr,visit_dt,op_cmpny_cd,visit_hr,visit_min
0,11680,710100004,2017-04-11,WMT-US,0,1
1,16237,710100004,2017-04-11,WMT-US,0,1
2,17365,710100004,2017-04-11,WMT-US,0,2
3,5071,710100005,2017-04-11,WMT-US,0,3
4,5443,710100005,2017-04-11,WMT-US,0,3


In [18]:
vst

,store_nbr,visit_nbr,visit_dt,op_cmpny_cd,visit_hr,visit_min
npartitions=4,,,,,,
,int64,int64,datetime64[ms],object,int16,int16
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


---
### Performing inner join between store_visit_scan and store_visit

In [19]:
%%time
# Q11. inner join the svs and vst tables on ['visit_dt', 'store_nbr', 'visit_nbr']
df = svs.merge(vst, on=['visit_dt', 'store_nbr', 'visit_nbr'], how='inner')

df = df.drop_duplicates()

CPU times: user 42.6 ms, sys: 15.7 ms, total: 58.3 ms
Wall time: 52.6 ms


In [20]:
df.head()

,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd_x,op_cmpny_cd_y,visit_hr,visit_min
334921,710100013,31,1,926596021,0,2017-06-23 12:10:28,2017-04-11,WMT-US,WMT-US,0,11
334923,710100013,31,2,885408808,0,2017-06-23 12:10:27,2017-04-11,WMT-US,WMT-US,0,11
334922,710100013,31,3,247820944,0,2017-06-23 12:10:27,2017-04-11,WMT-US,WMT-US,0,11
143788,710100013,106,1,399520165,0,2017-06-23 12:09:44,2017-04-11,WMT-US,WMT-US,0,21
237869,710100013,115,1,846065128,0,2017-06-23 12:10:10,2017-04-11,WMT-US,WMT-US,0,8


In [21]:
df

,visit_nbr,store_nbr,scan_seq_nbr,scan_id,scan_type,load_ts,visit_dt,op_cmpny_cd_x,op_cmpny_cd_y,visit_hr,visit_min
npartitions=1,,,,,,,,,,,
,int64,int64,int64,int64,int64,datetime64[ms],datetime64[ms],object,object,int16,int16
,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
## Q12. How many partitions is your df deivided into now?
## A. 

In [ ]:
## Q13. Change the datafarme(df) from previous number of partitons to to 4 partitions
df = df.<________>

---
#### Reading the item_dim table from `/data/item_dim_mask.orc` stored in orc format

In [ ]:
%%time
## Q14.  load the orc file as 6 partitons into dask dat frame
item = <____>('/data/item_dim_mask.orc').repartition(<_________>)


In [ ]:
item.head(), item

---
### Performing inner join with item_dim

In [ ]:
%%time
df = df.rename(columns = {'scan_id':'mds_fam_id'})

## Q15. merge df on item 
df = <_______>(item, on=['mds_fam_id'], how='inner')


In [ ]:
df.head(), df

In [ ]:
%%time
df = df.rename(columns = {'visit_dt':'cal_dt'})
## Q16. inner join df and cal on cal_dt 
df = df.<___________>

In [ ]:
df.head()

In [ ]:
df

In [ ]:
## Q17. How many partitions does your result df have?
## A. 

In [ ]:
## Q18. Persist the dataframe and make sure go and check your dask_graph in Bokeh link

df = df.<________>

### top 5 most sold items and their count

In [ ]:
most_selling_item_series = df['mds_fam_id'].value_counts().head(5)
most_selling_item_df = most_selling_item_series.to_frame("mds_fam_id_count")
most_selling_item_df

---
### get the other attributes corresponding to those items

In [ ]:
out_df = df.merge(most_selling_item_df, left_on='mds_fam_id', right_index=True, how='inner')
out_df.head()

In [ ]:
## Q19-20. Clean Up, Close the Client

cluster.<_____>
client.<_____>